In [20]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import pandas as pd
import numpy as np
from pprint import pprint

In [4]:
# formats to scrape, based on Bandcamp URL options
FORMATS = ['all',
           'digital',
           'vinyl',
           'cd',
           'cassette']

# search categories
#   top = best-selling
#   new = new arrivals
#   rec = artist-recommended
SEARCH_CATEGORIES = ['top','new','rec']

# scrape dataframe
scrape_data = pd.DataFrame(columns=['Genre',
                                    'Sub-Genre',
                                    'Release Name', 
                                    'Artist Name', 
                                    'Artist Location',
                                    'Release Date',
                                    'Track Information',
                                    'Fan Popularity Score',
                                    'Album Tags'])

In [6]:
# scrape bandcamp release page function
def scrape_release_page(URL):
    pass

In [21]:
# scrape search page function
def scrape_search_page(URL):

    #print(URL)
    response = requests.get(URL)
    
    if response.status_code == 200:
        print(URL)
        soup = bsoup(response.text, 'lxml')
        pprint(soup)

In [12]:
###################### INPUTS ################################

# list of genres to scrape
genres = ['all','rock']

# scrape for subgenres within genres?
scrape_subgenres = True

# dictionary of subgenres to scrape, if scrape_subgenres = True
SUBGENRES = {
                'rock': ['all','indie','prog-rock','post-rock','rock-roll','psychedelic-rock'],
                'metal': [],
                'alternative': [],
                'hip-hop-rap': [],
                'experimental': [],
                'punk': [],
                'folk': [],
                'pop': [],
                'acoustic': [],
                'funk': [],
                'country': [],
                'blues': []
            }

# list of search categories to scrape
search_categories = ['top']

# list of locations to scrape
# location = 0 returns search results for all locations
locations = [0] 

# list of formats to scrape
formats = ['all']

# number of pages to scrape
pages = 10 

###########################################################


In [22]:
# web scraper loop

for genre in genres:
    for category in search_categories:
        for location in locations:
            for formatt in formats:
                for page in range(pages):
                    
                    scrape_URL = 'https://bandcamp.com/?g=' + genre + '&s=' + category + '&p=' + str(page) + '&gn=' + str(location) + '&f=' + formatt
                    scrape_search_page(scrape_URL)
                    break
                    
                    if scrape_subgenres and genre in SUBGENRES:
                        for subgenre in SUBGENRES[genre]:
                            subgenre_scrape_URL = scrape_URL + '&t=' + subgenre
                            scrape_search_page(subgenre_scrape_URL)           

https://bandcamp.com/?g=all&s=top&p=0&gn=0&f=all
<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title>Bandcamp</title>
<meta content="
Discover amazing music and directly support the artists who make it.
" name="description"/>
<link href="https://s4.bcbits.com/img/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://s4.bcbits.com/img/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://s4.bcbits.com/img/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link color="#1da0c3" href="https://s4.bcbits.com/img/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="#603cba" name="msapplication-TileColor"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="website" property="og:type"/>
<meta content="https://bandcamp.com" property="og:url"/>
<met

https://bandcamp.com/?g=rock&s=top&p=0&gn=0&f=all
<!DOCTYPE html>
<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<title>Bandcamp</title>
<meta content="
Discover amazing music and directly support the artists who make it.
" name="description"/>
<link href="https://s4.bcbits.com/img/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="https://s4.bcbits.com/img/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="https://s4.bcbits.com/img/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link color="#1da0c3" href="https://s4.bcbits.com/img/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
<meta content="#603cba" name="msapplication-TileColor"/>
<meta content="#ffffff" name="theme-color"/>
<meta content="yes" name="apple-mobile-web-app-capable"/>
<meta content="website" property="og:type"/>
<meta content="https://bandcamp.com" property="og:url"/>
<me